# Data Base

Florian EPAIN
20105403, MASTER IL group 3

## Belatedly Installation

We can visualize blank `.csv` file using `vscode` and the extension `Rainbow CSV`.

### Docs

- I used the [Psycopg2 tutorials](https://www.psycopg.org/docs/install.html#quick-install) to connect my notebook to my database.
- [SQL Datatypes](https://www.w3schools.com/sql/sql_datatypes.asp)

In [ ]:
!pip install -U pip

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.6 MB/s eta 0:00:00m eta 0:00:010:01


In [ ]:
!pip install duckdb==0.9.1

In [1]:
# If the Server postgres is down
import duckdb
cursor = duckdb.connect()
# print(cursor.execute('SELECT 42').fetchall())

In [1]:
import psycopg2
from getpass import getpass
  
# Connection establishment 
conn = psycopg2.connect( 
    database="fepain", 
    user='fepain', 
    password=getpass(), 
    host='chausey.istic.univ-rennes1.fr'
)
  
conn.autocommit = True
  
# Creating a cursor object 
cursor = conn.cursor()

········


In [9]:
# Query to create a table 
sql = ''' CREATE TABLE TripData 
( 
    medallion VARCHAR(32), 
    hack_licence VARCHAR(32), 
    vendor_id VARCHAR(10), 
    rate_code INT,
    store_and_fwd_flag VARCHAR(10),
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count INT,
    trip_time_in_secs INT,
    trip_distance FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT
); '''
  
# --- Executing above query ---
cursor.execute(sql) # duckdb & postgres
print("Table has been created successfully D:")

Table has been created successfully D:


---

We now import the `csv` file, using the [Cursor Copy - tutorials](https://www.psycopg.org/docs/cursor.html#cursor.copy_from) and the [COPY docs](https://www.postgresql.org/docs/current/sql-copy.html). I can't use `copy_from()`, I will use `copy_expert()` as mentionned in this [github issue](https://github.com/psycopg/psycopg2/issues/1294)

---

In [6]:
# Display the table 
cursor.execute('SELECT * FROM TripData')
print(cursor.fetchall())

[]


In [2]:
# Import CSV in PostgreSQL
csv_file_path = "trip_data-sample.csv"
# cursor.copy_from(f, 'TripData')

with open(csv_file_path, 'r') as f:
    cursor.copy_expert(
    "COPY TripData FROM STDIN With CSV HEADER DELIMITER as ','",
    f)

# Display the table 
cursor.execute('SELECT * FROM TripData')
print(cursor.fetchall())

[('89D227B655E5C82AECF13C3F540D4CF4', 'BA96DE419E711691B9445D6A6307C170', 'CMT', 1, 'N', datetime.datetime(2013, 1, 1, 15, 11, 48), datetime.datetime(2013, 1, 1, 15, 18, 10), 4, 382, 1.0, -73.978165, 40.757977, -73.989838, 40.751171), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.006683, 40.731781, -73.994499, 40.75066), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.1, -74.004707, 40.73777, -74.009834, 40.726002), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786310', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 23, 54, 15), datetime.datetime(2013, 1, 7, 23, 58, 20), 2, 244, 0.7, -73.974602, 40.759945, -73.984734, 40.759388), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786

In [22]:
from sys import stdout

cursor.copy_expert("COPY TripData TO STDOUT WITH CSV HEADER", stdout)

medallion,hack_licence,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude


In [3]:
# Import Sample CSV in DuckDB

# cursor.execute('SELECT * FROM \'trip_data-sample.csv\';')
# print(cursor.fetchall())

sql = ''' COPY TripData FROM 'trip_data-sample.csv' With CSV HEADER DELIMITER as ','; '''
cursor.execute(sql)
print(cursor.fetchall())

[(9,)]


In [8]:
# Drop the table 
cursor.execute('DROP TABLE TripData')

---

Big CSV Import

---

In [11]:
# Import CSV in PostgreSQL
csv_file_path = "trip_data_1.csv"
# cursor.copy_from(f, 'TripData')

with open(csv_file_path, 'r') as f:
    cursor.copy_expert(
    "COPY TripData FROM STDIN With CSV HEADER DELIMITER as ','",
    f)

In [3]:
# Import Big CSV in DuckDB

# cursor.execute('SELECT * FROM \'trip_data_1.csv\';')
# print(cursor.fetchall())

sql = ''' COPY Trip FROM 'trip_data_1.csv' With CSV HEADER DELIMITER as ','; '''
cursor.execute(sql)
print(cursor.fetchall())

[(14776615,)]


# Analyze Queries


- [`EXPLAIN` docs](https://www.postgresql.org/docs/current/sql-explain.html)
- <https://explain.dalibo.com/>
- Use [`CLUSTER`](https://www.postgresql.org/docs/current/sql-cluster.html)
- Use [`CREATE INDEX name on Table (column_to_index)`](https://www.postgresql.org/docs/current/sql-createindex.html)

In [ ]:
# Optimize
optimize = 'CREATE INDEX index on TripData (medallion);'
cursor.execute(optimize)
print(cursor.fetchall())

print("-----------")

cluster = 'CLUSTER TripData using index;'
cursor.execute(cluster)
print(cursor.fetchall())

In [27]:
# Query 0
query = 'SELECT * FROM TripData WHERE medallion = \'0BD7C8F5BA12B88E0B67BED28BEA73D8\';'

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print("                                                     QUERY PLAN\n--------------------------------------------------------------------------------------------------------------------")
execution_plan = cursor.fetchall()
for line in execution_plan:
    print(line[0])
print("--------------------------------------------------------------------------------------------------------------------")

start_time = time.perf_counter()
number_of_iteration = 1;

print("                                                      QUERY 0")
for x in range(number_of_iteration):
    cursor.execute(query)
    # print(cursor.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
Bitmap Heap Scan on tripdata  (cost=17.50..4433.00 rows=1154 width=140)
  Recheck Cond: ((medallion)::text = '0BD7C8F5BA12B88E0B67BED28BEA73D8'::text)
  ->  Bitmap Index Scan on index  (cost=0.00..17.21 rows=1154 width=0)
        Index Cond: ((medallion)::text = '0BD7C8F5BA12B88E0B67BED28BEA73D8'::text)
--------------------------------------------------------------------------------------------------------------------
                                                      QUERY 0
Average Time Elapsed / sql execution (in sec):
0.05355804099235684


In [26]:
# Query 1
query = 'SELECT * FROM TripData WHERE passenger_count >= 2;'

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print("                                                     QUERY PLAN\n--------------------------------------------------------------------------------------------------------------------")
execution_plan = cursor.fetchall()
for line in execution_plan:
    print(line[0])
print("--------------------------------------------------------------------------------------------------------------------")

start_time = time.perf_counter()
number_of_iteration = 1;

print("                                                      QUERY 1")
for x in range(number_of_iteration):
    cursor.execute(query)
    # print(cursor.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
Seq Scan on tripdata  (cost=0.00..506058.69 rows=4308861 width=140)
  Filter: (passenger_count >= 2)
JIT:
  Functions: 2
  Options: Inlining true, Optimization true, Expressions true, Deforming true
--------------------------------------------------------------------------------------------------------------------
                                                      QUERY 1
Average Time Elapsed / sql execution (in sec):
9.418628131010337


In [25]:
# Query 2
query = 'SELECT * FROM TripData WHERE passenger_count = 1;'

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print("                                                     QUERY PLAN\n--------------------------------------------------------------------------------------------------------------------")
execution_plan = cursor.fetchall()
for line in execution_plan:
    print(line[0])
print("--------------------------------------------------------------------------------------------------------------------")

start_time = time.perf_counter()
number_of_iteration = 1;

print("                                                      QUERY 2")
for x in range(number_of_iteration):
    cursor.execute(query)
    # print(cursor.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
Seq Scan on tripdata  (cost=0.00..506058.69 rows=10467754 width=140)
  Filter: (passenger_count = 1)
JIT:
  Functions: 2
  Options: Inlining true, Optimization true, Expressions true, Deforming true
--------------------------------------------------------------------------------------------------------------------
                                                      QUERY 2
Average Time Elapsed / sql execution (in sec):
21.648114531009924


In [24]:
# Aggregate 1
aggregate = 'SELECT medallion, AVG(trip_distance) FROM TripData WHERE passenger_count = 1 GROUP BY medallion;'

explain = 'EXPLAIN ' + aggregate
cursor.execute(explain)
print("                                                     QUERY PLAN\n--------------------------------------------------------------------------------------------------------------------")
execution_plan = cursor.fetchall()
for line in execution_plan:
    print(line[0])
print("--------------------------------------------------------------------------------------------------------------------")

start_time = time.perf_counter()
number_of_iteration = 1;

print("                                                     AGGREGATE 1")
for x in range(number_of_iteration):
    cursor.execute(aggregate)
    # print(cursor.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
Finalize GroupAggregate  (cost=422117.32..425379.40 rows=12750 width=41)
  Group Key: medallion
  ->  Gather Merge  (cost=422117.32..425092.52 rows=25500 width=65)
        Workers Planned: 2
        ->  Sort  (cost=421117.29..421149.17 rows=12750 width=65)
              Sort Key: medallion
              ->  Partial HashAggregate  (cost=420120.36..420247.86 rows=12750 width=65)
                    Group Key: medallion
                    ->  Parallel Seq Scan on tripdata  (cost=0.00..398312.54 rows=4361564 width=41)
                          Filter: (passenger_count = 1)
JIT:
  Functions: 9
  Options: Inlining false, Optimization false, Expressions true, Deforming true
--------------------------------------------------------------------------------------------------------------------
                         

In [23]:
import time
import json

# Aggregate 2

aggregate = 'SELECT AVG(passenger_count) FROM TripData GROUP BY pickup_datetime;'

explain = 'EXPLAIN ' + aggregate
cursor.execute(explain)
print("                                                     QUERY PLAN\n--------------------------------------------------------------------------------------------------------------------")
execution_plan = cursor.fetchall()
for line in execution_plan:
    print(line[0])
print("--------------------------------------------------------------------------------------------------------------------")

start_time = time.perf_counter()
number_of_iteration = 1;

print("                                                     AGGREGATE 2")
for x in range(number_of_iteration):
    cursor.execute(aggregate)
    # print(cursor.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)



                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
Finalize GroupAggregate  (cost=856301.62..893361.70 rows=144851 width=40)
  Group Key: pickup_datetime
  ->  Gather Merge  (cost=856301.62..890102.55 rows=289702 width=40)
        Workers Planned: 2
        ->  Sort  (cost=855301.60..855663.73 rows=144851 width=40)
              Sort Key: pickup_datetime
              ->  Partial HashAggregate  (cost=777348.11..838922.82 rows=144851 width=40)
                    Group Key: pickup_datetime
                    Planned Partitions: 4
                    ->  Parallel Seq Scan on tripdata  (cost=0.00..382920.23 rows=6156923 width=12)
JIT:
  Functions: 7
  Options: Inlining true, Optimization true, Expressions true, Deforming true
--------------------------------------------------------------------------------------------------------------------
                   

DiskFull: ERREUR:  n'a pas pu écrire dans le fichier « base/pgsql_tmp/pgsql_tmp105552.1 » : Aucun espace disponible sur le périphérique
CONTEXT:  processus parallèle


### Comparison

- postgres noOpti 28.3206s
- duckDB noOpti 0.0017s

In [28]:
import time

# Aggregate 3

# Taxis' stats
query = 'SELECT medallion, AVG(passenger_count) as avg_pass_count, AVG(trip_distance) as avg_trip_dist FROM TripData GROUP BY medallion'
aggregate = 'SELECT avg_pass_count FROM (' + query + ') as taxisStats WHERE avg_trip_dist > (' + global_avg + ');'

explain = 'EXPLAIN ' + aggregate
cursor.execute(explain)
print("                                                     QUERY PLAN\n--------------------------------------------------------------------------------------------------------------------")
execution_plan = cursor.fetchall()
for line in execution_plan:
    print(line[0])
print("--------------------------------------------------------------------------------------------------------------------")

start_time = time.perf_counter()
number_of_iteration = 5;

print("                                                     AGGREGATE 3")
for x in range(number_of_iteration):
    cursor.execute(aggregate)
    # print(cursor.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)


NameError: name 'global_avg' is not defined

### Comparison

- postgres noOpti 6.0251s
- duckDB noOpti 0.0017s

## Compare DuckDB and PostgreSQL

- OLAP vs OLTP

In [13]:
# Closing the connection 
conn.close()